In [3]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()

/home/q123/Desktop/explo


In [4]:
env_name = "CartPole-v1"
kernel_name = "rbf"
#kernel_name = "linearstate"

env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
additional_layers=[] ### can be empty or [8,7] for adding 2 layers with width 8,7 respectively
model,objective_env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers)

trainer_config = {
        "n_steps" :200,
        "report_freq":5,
        "save_best":False,
}

### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
rslt= trainer.run()

MathLog.src.helpers : WARNING : MLP dimensions : [4, 1]
Using ard_num_dims = 5
 Gibo will use 20 last points to fit GP and 8 info samples
current 0.06000000610947609 / max 0.9999942183494568 /batch_mean 0.12559998035430908 /batch_max 0.2300001084804535 
##############################
covar_lengthscale max 0.33570021390914917 / min 0.32904568314552307                      covar_outputscale 0.01000006403774023                     noise 0.20000000298023224
##############################
last parameters tensor([-0.0381,  0.0579,  0.2133,  0.2232,  0.0942])
MLL : -0.2840518057346344
current 0.05800000578165054 / max 0.9999942183494568 /batch_mean 0.03320000320672989 /batch_max 0.05800000578165054 
##############################
covar_lengthscale max 0.33453798294067383 / min 0.29247167706489563                      covar_outputscale 0.01000006403774023                     noise 0.20000000298023224
##############################
last parameters tensor([0.1031, 0.0783, 0.2754, 0.3235, 0.1504]

KeyboardInterrupt: 